# Introducción

El presente documento resume un análisis comparativo respecto de las direcciones de productos, provistas por el area de GIS (XYGO) y el departamento de ventas Open Smart Flex (OSF).

- El objetivo es identificar los productos(direcciones) con mayor diferencia entre las direcciones XYGO - OSF, de manera que puedan ser revisadas por las áreas funcionales.

Los datos provienen de una base de datos ORACLE.
 -  Oracle Database 11g Enterprise Edition Release 11.2.0.4.0 - 64bit Production.
 
### Descripcion de los datos: 

|Descripción      | Dato     |  
|:----------------|---------:| 
|Identificador del producto|XYGO_PRODUCT_ID|   
|La dirección registrada en el proceso de venta (OSF)|OSF_ADDRESS|    
|La dirección entregada por área GIS (hipótesis de que XYGO sea la CORRECTA) |XYGO_ADDRESS|
|Nombre de calle entregado por área GIS |XYGO_WAY_NAME | 
|El N° de calle entregado por área GIS |XYGO_WAY_NUMBER | 
|Latitud |LAT| 
|Longitud |LON| 
|Variable binaria 0,1|SCORE_A| 
|Variable binaria 0,1|SCORE_B|  

### Extraccion de campos desde SQL Server
```  
SELECT      
		XYGO_PRODUCT_ID,   
		LAT,
		LON,
		XYGO_WAY_NUMBER,
		XYGO_WAY_NAME, 
		XYGO_ADDRESS, 						-- dirección sistema A (hipótesis de que XYGO sea la CORRECTA)
		AB_ADDRESS.ADDRESS "OSF_ADDRESS", 	-- dirección sistema B (OSF) 
CASE WHEN trim(XYGO_ADDRESS) = trim(AB_ADDRESS.ADDRESS) THEN 1 ELSE 0 end SCORE_A,
CASE WHEN AB_ADDRESS.ADDRESS LIKE '%'||XYGO_WAY_NUMBER||'%' THEN 1 ELSE 0 end SCORE_B  
FROM SFINTERFAZ.XYGO_DIRECCIONES
JOIN CHILQUIN.PR_PRODUCT@SFAA ON PR_PRODUCT.PRODUCT_ID = XYGO_DIRECCIONES.XYGO_PRODUCT_ID
JOIN CHILQUIN.AB_ADDRESS@SFAA ON AB_ADDRESS.ADDRESS_ID = PR_PRODUCT.ADDRESS_ID
```


In [1]:
library(sparklyr)
library(dplyr)  
library(ggplot2)
library(knitr)#tablas
library(ggplot2)
library(scales) # funcion de %
library(readr)
library(sp) 
library(leaflet)
library(plyr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'readr'

The following object is masked from 'package:scales':

    col_factor

------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: 'plyr'

The following objects are masked from 'package:dplyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



# Conexión a Spark R

In [2]:
rm(list=ls())  
sc <- spark_connect( master = "local" ) 

# Carga de datos csv

In [3]:
direcciones = spark_read_csv(sc, name = "direcciones", path = "XYGO_DIRECCIONES.csv" ,delimiter = ";")

In [4]:
head(direcciones)

# Source:   lazy query [?? x 9]
# Database: spark_connection
  XYGO_PRODUCT_ID LAT   LON   XYGO_WAY_NUMBER XYGO_WAY_NAME XYGO_ADDRESS
            <dbl> <chr> <chr>           <dbl> <chr>         <chr>       
1            101. -33,~ -71,~             167 AVENIDA DEL ~ AVENIDA DEL~
2            103. -33,~ -71,~              84 GENERAL MACK~ GENERAL MAC~
3            104. -33,~ -71,~             450 AVENIDA BORI~ AVENIDA BOR~
4            104. -33,~ -71,~             100 AVENIDA BORI~ AVENIDA BOR~
5            104. -33,~ -71,~              34 AVENIDA BORI~ AVENIDA BOR~
6            104. -33,~ -71,~             476 AVENIDA BORI~ AVENIDA BOR~
# ... with 3 more variables: OSF_ADDRESS <chr>, SCORE_A <int>, SCORE_B <int>

## Normalizacion de direcciones XYGO Y OSF
Como se menciona la normalizacion de palabras involucra remover espacios antes y despues de un texto  , los acentos , caracteres especiales , ademas de convertir a mayuscula o minuscula todas la palabras para minimizar la diferencia al momento de comparar. <br>

Se crearan nuevas columnas con las direcciones normalizadas agregandoles el sufijo _CLEAN.


In [5]:
## Cargamos los datos en memoria R
data = direcciones %>% collect()
  
## Removemos
## Dobles espacios  
data$XYGO_ADDRESS_CLEAN     =  gsub("[[:punct:]]", " ",  data$XYGO_ADDRESS)
data$XYGO_WAY_NAME_CLEAN    =  gsub("[[:punct:]]", " ",  data$XYGO_WAY_NAME)
data$OSF_ADDRESS_CLEAN            =  gsub("[[:punct:]]", " ",  data$OSF_ADDRESS)

## Caracteres no alfanumericos ;
data$XYGO_ADDRESS_CLEAN     =  gsub("[^[:alnum:]///' ]", " ", data$XYGO_ADDRESS_CLEAN)
data$XYGO_WAY_NAME_CLEAN    =  gsub("[^[:alnum:]///' ]", " ", data$XYGO_WAY_NAME_CLEAN)
data$OSF_ADDRESS_CLEAN            =  gsub("[^[:alnum:]///' ]", " ", data$OSF_ADDRESS_CLEAN)

## Caracteres especiales
data$XYGO_ADDRESS_CLEAN     =  gsub("[^[:alnum:]///' ]+^ *|(?<= ) | *$", "", data$XYGO_ADDRESS_CLEAN, perl = TRUE)
data$XYGO_WAY_NAME_CLEAN    =  gsub("[^[:alnum:]///' ]+^ *|(?<= ) | *$", "", data$XYGO_WAY_NAME_CLEAN, perl = TRUE)
data$OSF_ADDRESS_CLEAN      =  gsub("[^[:alnum:]///' ]+^ *|(?<= ) | *$", "", data$OSF_ADDRESS_CLEAN, perl = TRUE)

In [7]:
head(data)

XYGO_PRODUCT_ID,LAT,LON,XYGO_WAY_NUMBER,XYGO_WAY_NAME,XYGO_ADDRESS,OSF_ADDRESS,SCORE_A,SCORE_B,XYGO_ADDRESS_CLEAN,XYGO_WAY_NAME_CLEAN,OSF_ADDRESS_CLEAN
100.750,"-33,03353277","-71,64074554",167,AVENIDA DEL PACIFICO,AVENIDA DEL PACIFICO 167B,AV PACIFICO 167 ESC,0,1,AVENIDA DEL PACIFICO 167B,AVENIDA DEL PACIFICO,AV PACIFICO 167 ESC
103.051,"-33,04633926","-71,6245623",84,GENERAL MACKENNA,GENERAL MACKENNA 84,PASAJE ECUADOR N° 69 CASA 3,0,0,GENERAL MACKENNA 84,GENERAL MACKENNA,PASAJE ECUADOR N 69 CASA 3
103.569,"-33,04798053","-71,56614281",450,AVENIDA BORINQUEN,AVENIDA BORINQUEN 450,BORINQUEN 450 D/504,0,1,AVENIDA BORINQUEN 450,AVENIDA BORINQUEN,BORINQUEN 450 D 504
103.598,"-33,04810073","-71,5653579",100,AVENIDA BORINQUEN,AVENIDA BORINQUEN 100,BORINQUEN 100 B/3 D/303,0,1,AVENIDA BORINQUEN 100,AVENIDA BORINQUEN,BORINQUEN 100 B 3 D 303
103.628,"-33,04745301","-71,56550569",34,AVENIDA BORINQUEN,AVENIDA BORINQUEN 34,BORINQUEN 34 B/5 D/101,0,1,AVENIDA BORINQUEN 34,AVENIDA BORINQUEN,BORINQUEN 34 B 5 D 101
103.658,"-33,04783461","-71,56565327",476,AVENIDA BORINQUEN,AVENIDA BORINQUEN 476,BORINQUEN 476 D/402,0,1,AVENIDA BORINQUEN 476,AVENIDA BORINQUEN,BORINQUEN 476 D 402
